In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
from google.colab import drive, files

In [ ]:
from itertools import repeat

In [ ]:
## Import data 

In [ ]:
match_results = pd.read_csv("/content/drive/MyDrive/projects/worldcup winner prediction/datasets/archive (1).zip (Unzipped Files)/results.csv")

In [ ]:
ranking = pd.read_csv("/content/drive/MyDrive/projects/worldcup winner prediction/datasets/fifa_ranking-2022-10-06.csv")

In [ ]:
## Structural analysis

In [ ]:
match_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


In [ ]:
match_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44206 entries, 0 to 44205
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        44206 non-null  object 
 1   home_team   44206 non-null  object 
 2   away_team   44206 non-null  object 
 3   home_score  44178 non-null  float64
 4   away_score  44178 non-null  float64
 5   tournament  44206 non-null  object 
 6   city        44206 non-null  object 
 7   country     44206 non-null  object 
 8   neutral     44206 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB


In [ ]:
ranking.head(10)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
1,96,Syria,SYR,11.0,0.0,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.0,0.0,0,CAF,1992-12-31
3,99,Latvia,LVA,10.0,0.0,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.0,0.0,0,CAF,1992-12-31
5,101,Togo,TOG,10.0,0.0,0,CAF,1992-12-31
6,102,Angola,ANG,10.0,0.0,0,CAF,1992-12-31
7,95,Suriname,SUR,12.0,0.0,0,CONCACAF,1992-12-31
8,103,Luxembourg,LUX,10.0,0.0,0,UEFA,1992-12-31
9,105,St. Vincent and the Grenadines,VIN,9.0,0.0,0,CONCACAF,1992-12-31


In [ ]:
ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63916 entries, 0 to 63915
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rank             63916 non-null  int64  
 1   country_full     63916 non-null  object 
 2   country_abrv     63916 non-null  object 
 3   total_points     63916 non-null  float64
 4   previous_points  63916 non-null  float64
 5   rank_change      63916 non-null  int64  
 6   confederation    63916 non-null  object 
 7   rank_date        63916 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 3.9+ MB


In [ ]:
ranking.shape


(63916, 8)

In [ ]:
ranking["rank_date"] = pd.to_datetime(ranking["rank_date"])

In [ ]:
match_results["date"] = pd.to_datetime(match_results["date"])

In [ ]:
##Filtering useful information

In [ ]:
match_results = match_results[match_results["date"] >= "2018-01-01"].reset_index(drop=True)

In [ ]:
ranking= ranking[ranking["rank_date"] >= "2018-01-01"].reset_index(drop=True)

In [ ]:
match_results1 = match_results.copy()
ranking1 = ranking.copy()

In [ ]:
## cleaning 

In [ ]:
ranking1["country_full"] = ranking1["country_full"].str.replace("IR Iran", "Iran").str.replace("Korean Rpeublic", "South Korea").str.replace("USA", "United State")

In [ ]:
## merge

In [ ]:
ranking1=ranking1.set_index(["rank_date"]).groupby(["country_full"], group_keys = False).resample("D").fillna(method = "ffill").reset_index()

In [ ]:
relevant_info = ["country_full", "total_points", "rank_date", "rank"]

In [ ]:
df_merge= match_results1.merge(ranking1[relevant_info], left_on = ["date", "home_team"], right_on = ["rank_date", "country_full"]).drop(["rank_date", "country_full"], axis=1)

In [ ]:
df_merge

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points,rank
0,2018-01-18,Uganda,Namibia,0.0,1.0,African Nations Championship,Marrakech,Morocco,True,446.00,73
1,2018-01-19,Libya,Nigeria,0.0,1.0,African Nations Championship,Tangier,Morocco,True,389.00,89
2,2018-01-19,Rwanda,Equatorial Guinea,1.0,0.0,African Nations Championship,Tangier,Morocco,True,282.00,116
3,2018-01-20,Angola,Cameroon,1.0,0.0,African Nations Championship,Agadir,Morocco,True,200.00,142
4,2018-01-20,Congo,Burkina Faso,2.0,0.0,African Nations Championship,Agadir,Morocco,True,369.00,96
...,...,...,...,...,...,...,...,...,...,...,...
3783,2022-09-27,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,False,1488.57,36
3784,2022-09-27,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,False,1563.44,20
3785,2022-09-27,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,False,1183.90,106
3786,2022-09-27,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,False,1441.45,49


In [ ]:
df_merge= df_merge.merge(ranking1[relevant_info], left_on = ["date", "away_team"], right_on = ["rank_date", "country_full"], suffixes = ("_home", "_away")).drop(["rank_date", "country_full"], axis=1)
df_merge


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points_home,rank_home,total_points_away,rank_away
0,2018-01-18,Uganda,Namibia,0.0,1.0,African Nations Championship,Marrakech,Morocco,True,446.00,73,300.00,111
1,2018-01-19,Libya,Nigeria,0.0,1.0,African Nations Championship,Tangier,Morocco,True,389.00,89,651.00,51
2,2018-01-19,Rwanda,Equatorial Guinea,1.0,0.0,African Nations Championship,Tangier,Morocco,True,282.00,116,202.00,141
3,2018-01-20,Angola,Cameroon,1.0,0.0,African Nations Championship,Agadir,Morocco,True,200.00,142,707.00,45
4,2018-01-20,Congo,Burkina Faso,2.0,0.0,African Nations Championship,Agadir,Morocco,True,369.00,96,709.00,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3554,2022-09-27,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,False,1488.57,36,1549.53,25
3555,2022-09-27,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,False,1563.44,20,1372.48,65
3556,2022-09-27,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,False,1183.90,106,1180.52,108
3557,2022-09-27,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,False,1441.45,49,1399.10,58


In [ ]:
##Target definition

In [ ]:
df_merge.loc[df_merge["home_score"] > df_merge["away_score"], "target"] = 1
df_merge.loc[df_merge["home_score"] == df_merge["away_score"], "target"] = 2
df_merge.loc[df_merge["home_score"] > df_merge["away_score"], "target"] = 3

In [ ]:
##Feature engineering

In [ ]:
df_merge


In [ ]:
##rank difference

In [ ]:
df_merge["rank_difference"] = df_merge["rank_home"] - df_merge["rank_away"]

In [ ]:
df_merge["rank_difference"]

0      -38
1       38
2      -25
3       97
4       52
        ..
3554    11
3555   -45
3556    -2
3557    -9
3558    26
Name: rank_difference, Length: 3559, dtype: int64

In [ ]:
## racha

In [ ]:
df_merge.sort_values(["home_team", "date"], inplace =True)

In [ ]:
df_merge

In [ ]:
df_merge["racha_home1"] = df_merge.groupby("home_team")["home_score"].shift(1)

In [ ]:
## Final cleanup

In [ ]:
df_merge.head()

In [ ]:
df_merge.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral', 'total_points_home',
       'rank_home', 'total_points_away', 'rank_away', 'target',
       'rank_difference', 'racha_home1'],
      dtype='object')

In [ ]:
final_variable= ['home_team', 'away_team','total_points_home',
       'rank_home', 'target', 'total_points_away', 'rank_away',
       'rank_difference', 'racha_home1']

In [ ]:
df_final = df_merge[final_variable]

In [ ]:
df_enconde_home= pd.get_dummies(df_final["home_team"], prefix = "_home")

In [ ]:
df_enconde_away= pd.get_dummies(df_final["home_team"], prefix = "_away")

In [ ]:
df_final1= pd.concat([df_final, df_enconde_home], axis= 1)

In [ ]:
df_final1.drop(["home_team", "away_team"], axis = 1, inplace = True)

In [ ]:
df_final1

In [ ]:
##nulos

In [ ]:
df_final1[df_final1.duplicated()]

In [ ]:
df_final1[df_final1.isna()]

In [ ]:
df_final1.dropna(inplace=True)

In [ ]:
##Modelicación

In [ ]:
df_final1

,total_points_home,rank_home,target,total_points_away,rank_away,rank_difference,racha_home1,_home_Afghanistan,_home_Albania,_home_Algeria,...,_home_United Arab Emirates,_home_Uruguay,_home_Uzbekistan,_home_Vanuatu,_home_Venezuela,_home_Vietnam,_home_Wales,_home_Yemen,_home_Zambia,_home_Zimbabwe
1287,1058.00,149,3.0,922.00,182,-33,0.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1513,1054.00,149,2.0,1201.00,106,43,1.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3409,1049.77,150,2.0,966.61,171,-21,1.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
359,1384.00,58,3.0,1256.00,93,-35,1.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
448,1383.00,57,2.0,1189.00,110,-53,1.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,1165.00,118,3.0,1078.00,137,-19,3.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1519,1167.00,117,2.0,1057.00,146,-29,3.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2494,1171.88,108,2.0,1337.77,73,35,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2817,1147.34,118,2.0,1079.37,137,-19,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
y = df_final1[["target"]]
y

,target
1287,3.0
1513,2.0
3409,2.0
359,3.0
448,2.0
...,...
1312,3.0
1519,2.0
2494,2.0
2817,2.0


In [ ]:
X= df_final1.drop("target", axis = 1)
X

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

In [ ]:
model = RandomForestClassifier(random_state = 3, max_depth= 9, n_estimators = 100)

In [ ]:
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(max_depth=9, random_state=3)

In [ ]:
## Match prediction

In [ ]:
model.predict(X_test)

In [ ]:
y_test_predict = pd.DataFrame(model.predict(X_test), index= y_test.index, columns= ["prediction"])

In [ ]:
pred = pd.concat((y_test, y_test_predict), axis= 1)

In [ ]:
pred["exito"] = (pred["target"] == pred["prediction"]).astype(int)

In [ ]:
pred

,target,prediction,exito
1994,3.0,3.0,1
1512,3.0,3.0,1
270,3.0,3.0,1
363,2.0,3.0,0
3426,3.0,3.0,1
...,...,...,...
1244,3.0,3.0,1
446,3.0,3.0,1
943,3.0,3.0,1
2495,3.0,3.0,1


In [ ]:
pred["exito"].mean()

0.6743215031315241

In [ ]:
## Predicciones mundial

In [ ]:
['total_points_home','rank_home',  'total_points_away', 'rank_away','rank_difference', 'racha_home1']

['total_points_home',
 'rank_home',
 'total_points_away',
 'rank_away',
 'rank_difference',
 'racha_home1']

In [ ]:
len(X_test.columns)

205

In [ ]:
game_mexico_poland= [1645, 13, 1549, 26, -13, 1]

In [ ]:
game_mexico_poland4= game_mexico_poland + list(repeat(1,199))





In [ ]:
wc_prediction = pd.DataFrame( [game_mexico_poland4], columns = X_test.columns)

In [ ]:
wc_prediction["_home_Mexico"] = 1

In [ ]:
wc_prediction["_away_Poland"] = 0

0    False
dtype: bool

In [ ]:
model.predict_proba(wc_prediction)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- _away_Poland
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


ValueError: ignored